In [2]:
#import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import xgboost as xgb
import os
import string

In [3]:
os.chdir('C:\\Users\\royal\\Downloads\\Compressed\\data\\Toxic Comment chal')

In [4]:
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [5]:
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer
import re
import string
import nltk
from nltk.corpus import stopwords

In [6]:
tokenizer=TweetTokenizer()

In [7]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
subm = pd.read_csv('sample_submission.csv')

In [8]:
#merge into single pandas object
merge=pd.concat([train.iloc[:,0:2],test.iloc[:,0:2]])

In [9]:
corpus=merge['comment_text']

In [10]:
corpus = corpus.str.lower()

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [22]:
#replace the Appostrophes,smileys and some commonly misspelled words.
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "i would",
"i'd" : "i had",
"i'll" : "i will",
"i'm" : "i am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "i have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

repl = {
    "&lt;3": " good ",
    ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
    "wtf":"what the fuck",
    "f ck":"fuck",
    "fucka":"fuck",
    "sh t":"shit",
    "fucky":"fuck",
    "mothafuc":"mothafucker",
    "fucksex":"fuck sex",
    "yourselfgo":"yourself go",
    "criminalwar":"criminal war",
    "securityfuck":"security fuck",
    "ancestryfuck":"ancestry fuck",
    "phuq":"fuck",
    "slimvirgin":"slim virgin",
    "itsuck":"it suck",
    "talk2me":"talk to me",
    "fuckingabf":"fucking",
    "fuckbags":"fuck bags",
    "failepic":"fail epic",
    'a n a l': 'anal',
 'a n u s': 'anus',
 'a r s e': 'arse',
 'a s s': 'ass',
 'animalfucker': 'animal fucker',
 'assclowns': 'ass clowns',
 'assface': 'ass face',
 'asskicked': 'ass kicked',
 'asswhipe': 'ass whipe',
 'b a l l s': 'balls',
 'b a l l s a c k': 'ballsack',
 'b a s t a r d': 'bastard',
 'b i a t c h': 'bitch',
 'b i t c h': 'bitch',
 'b l o o d y': 'bloody',
 'b l o w j o b': 'blowjob',
 'b o l l o c k': 'bollock',
 'b o l l o k': 'bollok',
 'b o n e r': 'boner',
 'b o o b': 'boob',
 'b u g g e r': 'bugger',
 'b u m': 'bum',
 'b u t t': 'butt',
 'b u t t p l u g': 'buttplug',
 'badassness': 'bad ass',
 'bicth': 'bitch',
 'bitchass': 'bitch ass',
 'bitchmattythewhite': 'bitch matty the white',
 'bitchmother': 'bitch mother',
 'bitchs': 'bitch',
 'blowjob': 'blow job',
 'boymamas': 'boy mamas',
 'c l i t o r i s': 'clitoris',
 'c o c k': 'cock',
 'c o o n': 'coon',
 'c r a p': 'crap',
 'c u n t': 'cunt',
 'corpsefucking': 'corpse fucking',
 'cuntbag': 'cunt bag',
 'cuntface': 'cunt face',
 'cuntfranks': 'cunt franks',
 'cuntliz': 'cunt liz',
 'd a m n': 'damn',
 'd i c k': 'dick',
 'd i l d o': 'dildo',
 'dickbag': 'dick bag',
 'dickbig': 'dick big',
 'dickbreath': 'dick breath',
 'dickbutt': 'dick butt',
 'dickheaditalic': 'dick head italic',
 'f a g': 'fag',
 'f e c k': 'feck',
 'f e l c h i n g': 'felching',
 'f e l l a t e': 'fellate',
 'f e l l a t i o': 'fellatio',
 'f l a n g e': 'flange',
 'f u c k': 'fuck',
 'f u d g e p a c k e r': 'fudge packer',
 'failepic': 'fail epic',
 'fatmansanger': 'fat mananger',
 'fcken': 'fuck',
 'fckin': 'fuck',
 'fcking': 'fuck',
 'fking': 'fuck',
 'fonkin': 'fuck',
 'fuck': 'fuck',
 'fuckan': 'fucking',
 'fuckass': 'fuck ass',
 'fuckbags': 'fuck bags',
 'fuckedy': 'fuck',
 'fuckhole': 'fuck hole',
 'fuckiest': 'fuck',
 'fuckign': 'fuck',
 'fuckingabf': 'fuck',
 'fuckk': 'fuck',
 'fuckon': 'fuck',
 'fucksex': 'fuck sex',
 'fuckstick': 'fuck stick',
 'fuckwads': 'fuck',
 'fuckyourself': 'fuck yourself',
 'fudgepacker': 'fudge packer',
 'fukcing': 'fuck',
 'fuked': 'fuck',
 'fuking': 'fuck',
 'fukkers': 'fuck',
 'fukyou': 'fuck you',
 'g o d d a m n': 'god damn',
 'goddamn': 'god damn',
 'h e l l': 'hell',
 'h o m o': 'homo',
 'i d i o t': 'idiot',
 'itiot': 'idiot',
 'itsuck': 'it suck',
 'j e r k': 'jerk',
 'j i z z': 'jizz',
 'jpgsuck': 'jpg suck',
 'k n o b e n d': 'knob end',
 'kickwars': 'kick wars',
 'knobend': 'knob end',
 'l a b i a': 'labia',
 'l m a o': 'lmao',
 'l m f a o': 'lmfao',
 'm u f f': 'muff',
 'marcolfuck': 'marcol fuck',
 'mothafuckin': 'mother fuck',
 'motherfu': 'mother fuck',
 'n i g g a': 'nigga',
 'n i g g e r': 'nigger',
 'niggertard': 'nigger tard',
 'o m g': 'omg',
 'ofuck': 'oh fuck',
 'oldlady': 'old lady',
 'p e n i s': 'penis',
 'p i s s': 'piss',
 'p o o p': 'poop',
 'p r i c k': 'prick',
 'p u b e': 'pube',
 'p u s s y': 'pussy',
 'penistown': 'penis',
 'pensnsnniensnsn': 'penis',
 'pneis': 'penis',
 'popsucker': 'pop sucker',
 'q u e e r': 'queer',
 's c r o t u m': 'scrotum',
 's e x': 'sex',
 's h 1 t': 'shit',
 's h i t': 'shit',
 's l u t': 'slut',
 's m e g m a': 'smegma',
 's p u n k': 'spunk',
 's u c k': 'suck',
 'sexcual': 'sexual',
 'sexsex': 'sex',
 'shiot': 'shit',
 'shioty': 'shit',
 'shitdip': 'shit',
 'shitfuck': 'shit fuck',
 'shitler': 'shit',
 'shitlol': 'shit',
 'shitush': 'shit',
 'shoit': 'shit',
 'suckdickeer': 'suck dick',
 'suckersyou': 'suck you',
 'suckipedia': 'suck',
 'suckish': 'suck',
 'sucksfrozen': 'suck frozen',
 't i t': 'tit',
 't o s s e r': 'tosser',
 't u r d': 'turd',
 't w a t': 'twat',
 'v a g i n a': 'vagina',
 'vbutt': 'butt',
 'w a n k': 'wank',
 'w a n k e r': 'wanker',
 'w h o r e': 'whore',
 'w t f': 'wtf',
 'wikifuckers': 'fuckers',
 'wikihomosexuals': 'homosexual',
 'wikijews': 'jew',
 'wikipedidiots': 'idiot',
 'wikipedophiles': 'pedophile',
 'wikiretards': 'retard',
 'wikisucks': 'suck',
 'wikitheclown': 'clown',
 'wikiwankers': 'wanker'
}

In [23]:
replkeys = [i for i in repl.keys()]

In [51]:
def clearup(s, chars):
    return re.sub('[%s]' % chars, '', s).lower()

In [86]:
#remove the punctuation from the text.
punctu=['"', '%', '&', "'", '(', ')', '+', '-', '/', ':', ';', '<', '=', '>', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']
def remove_punctuations(text):
    for punctuation in punctu:
        text = text.replace(punctuation, ' ')
        text = ' '.join(text.split())
    return text

In [87]:
"""
This function receives comments and returns clean word-list
"""
def clean(comment):
    #Convert to lower case , so that Hi and hi are the same
    #comment=comment.lower()
    #remove \n
    comment=re.sub("\\n"," ",comment)
    # remove leaky elements like ip,user
    comment=re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"," ",comment)
    #removing usernames
    comment=re.sub("\[\[.*\]"," ",comment)
    #remove hyperlink
    comment=re.sub("http\S+|www.\S+"," ",comment)
    #remove numbers
    comment = clearup(comment, string.punctuation+string.digits)
    #Split the sentences into words
    words=tokenizer.tokenize(comment)
    # (')aphostophe  replacement (ie)   you're --> you are  
    # ( basic dictionary lookup : master dictionary present in a hidden block of code)
    words=[APPO[word] if word in APPO else word for word in words]
    #other commonly misspeled words.
    words=[repl[word] if word in replkeys else word for word in words]
    clean_sent=" ".join(words)
    # remove any non alphanum,digit character
    clean_sent=re.sub("\W+"," ",clean_sent)
    # remove the punctuations from the text
    remove_punctuations(clean_sent)
    clean_sent=re.sub("  "," ",clean_sent)
    clean_sent=clean_sent.strip()
    return(clean_sent)

In [43]:
#corpus.iloc[corpus.str.contains('suckish')]
corpus[corpus.str.contains("wasn't")]
#.str.contains('Mel')

45        i don't believe the lisak criticism present th...
73        "\n ambiguous ? \nso mabuska irish can mean mo...
193       about mitch moved to yggdrasill \n\ni have no ...
249       "\nit's in the history, or more conveniently i...
404       hi collectonian, it obviously wasn't appropria...
454       "\ni want to offer you a tremendous ""thank yo...
484       "== tfd nomination of template:silentredirect ...
492       it wasn't me that added the glebe park link, t...
538       i'm disappointed it wasn't a successful run, b...
553       ipad talk page \n\nwhy did you remove that ip'...
557       "\n\nnope, i didn't read the guide to appealin...
791       referring to her as *catherine* \n\ni was told...
955       ":you said: ""i am determined that the page wi...
1051      thanks to all for taking my rather offhand com...
1130      ha! \n\nthere wasn't a blp violation in the co...
1164             your edit summary wasn't very nice either.
1231      "\n\n hello \n\nhi blu aardvar

In [91]:
#apply the clean method on first 200 comments
clean_corpus200=corpus[:200].apply(lambda x :clean(x))

In [88]:
%%time
clean_corpus=corpus.apply(lambda x :clean(x))

Wall time: 9min 27s


In [85]:
clean_corpus[clean_corpus.str.contains("shit fuck")]

6346      riaa certifications someone got that shit fuck...
14025     hola fucker retard who has nothing better to d...
15469     whatthefuck what the fuck dude i cant add my o...
18199                   you better fuck off ruddy shit fuck
30737     hey you know what fuck this it takes to goddam...
31680     nuisance piece of shit fuck off wiki doesnt ne...
32814     just this account dipshit fuck are you dumb a ...
36213     i feel the page should not be deleted as you g...
40597     fuck you you piece of shit fuck you you piece ...
40941                   you better fuck off ruddy shit fuck
41390     she is a motherfucker i will block you motherf...
42919     eat shit fucko suck my asshole then die horrib...
52456     chestop adding in shit only to support your fu...
61304        you cant delete it siece of shit fuck your mom
62768     suck it bitch suck my bigass cock gag on it mo...
66463     fuck you niggerkite y you deleting so much shi...
71866     i will find you in real life a

In [89]:
corpus.iloc[113]

"thanks, josette. i enjoyed meeting you, too. i was shocked by the decision, which does not begin to reflect consensus. does just one grand poobah make it alone? serves me right for stealing time from more pressing real-world duties to indulge in a fun hobby. i've learned my lesson and won't waste time like that again. i'll stick to fixing the little things i run across as i read articles for my own information."

In [92]:
clean_corpus200.iloc[113]

'thanks josette i enjoyed meeting you too i was shocked by the decision which does not begin to reflect consensus does just one grand poobah make it alone serves me right for stealing time from more pressing realworld duties to indulge in a fun hobby ive learned my lesson and wont waste time like that again ill stick to fixing the little things i run across as i read articles for my own information'

In [90]:
clean_corpus.iloc[113]

'thanks josette i enjoyed meeting you too i was shocked by the decision which does not begin to reflect consensus does just one grand poobah make it alone serves me right for stealing time from more pressing realworld duties to indulge in a fun hobby ive learned my lesson and wont waste time like that again ill stick to fixing the little things i run across as i read articles for my own information'

In [ ]:
##check the other language characters in the comment.
ascii = set(string.printable)
def remove_non_ascii(s):
    return list(filter(lambda x: x not in ascii, s))

In [ ]:
# corp_ascii=corpus.apply(lambda x:remove_non_ascii(x))
clean_corpus_ascii=clean_corpus.apply(lambda x:remove_non_ascii(x))
# corp_ascii=corpus.apply(lambda x: x not in ascii)

In [ ]:
clean_corpus[:5]

In [ ]:
k={}
for i,j in enumerate(clean_corpus_ascii):
    if (len(j)!=0):
        k[i]=j
#         k.append(j)

In [ ]:
p={}
for i,j in k.items():
    if len(j)>10:
        p[i]=j
        #print(len(i))
        #p.append(i)

In [ ]:
##check the other language containing comments.
# 8912 are contain non-ascii characrters
len(list(k.values()))
#1960 words contain other language text
len(list(p.values()))
list(p.keys())[:50]

In [ ]:

clean_corpus.iloc[24515]
clean_corpus.iloc[22254]

In [ ]:
train['comment']=clean_corpus[:train.shape[0]]
test['comment']=clean_corpus[train.shape[0]:]

In [ ]:
columnsTitles=['id', 'comment','toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']
train1=train.reindex(columns=columnsTitles)
train1.columns = ['id', 'comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate']
train1.to_csv('train_clean.csv',encoding = "utf8", index=False)

columnsTitles=['id', 'comment']
test1=test.reindex(columns=columnsTitles)
test1.columns=['id', 'comment_text']
test1.to_csv('test_clean.csv',encoding = "utf8", index=False)